In [1]:
import deepchem as dc
from sklearn.metrics import mean_squared_error
import numpy as np

2024-10-07 20:45:04.653751: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-07 20:45:04.664286: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 20:45:04.676445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-07 20:45:04.679948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-07 20:45:04.689712: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [ ]:
import pyprithvi
pyprithvi.set_backend_url('https://cb.deepforestsci.com/')
pyprithvi.healthcheck()

# Please login to Prithvi
pyprithvi.login(username='', password='')
pyprithvi.set_session_profile("LLNL")
pyprithvi.set_session_project('bench_10M_mol_token_e5')

In [10]:
dataset = 'delaney'
# dataset = 'regressBACE'
# dataset = 'clearance'
# dataset = 'lipo'

In [12]:
for learning_rate in [1e-05]:
	for batch_size in [15]:
		for nb_epoch in [50]:
			model_key = f'finetune-chemberta-zinc10m-molf-e5-{dataset}-batch{batch_size}-epoch{nb_epoch}-lr{learning_rate}'
			response = pyprithvi.train(
				dataset_address=f'chiron://deep-forest-sciences/moleculenet/feat_{dataset}_finetune_train',
				model_type='chemberta',
				model_key= model_key,
				init_kwargs={'learning_rate': learning_rate, 'batch_size': batch_size},
				train_kwargs={'nb_epoch': nb_epoch},
				pretrained_model_address="chiron://LLNL/models/chemberta_10m_mlm_molf_token_5e",
				task='regression',
				job_config_id='cjc-13')

			print(response)


{'job_id': '44d3194c-b575-43fc-99f7-e6bd4b8807ab', 'operation_cost': 0.0416724}


In [14]:
# for bace dataset use dataset_column = 'mol'

for data in ["train", "test", "valid"]:
    for learning_rate in [1e-05]:
        for batch_size in [15]:
            for nb_epoch in [50]:
                    response = pyprithvi.infer(
                        model_address=f'finetune-chemberta-zinc10m-molf-e5-{dataset}-batch{batch_size}-epoch{nb_epoch}-lr{learning_rate}',
                        dataset_address=f"chiron://deep-forest-sciences/moleculenet/feat_{dataset}_finetune_{data}",
                        output_key=f"chiron://LLNL/bench_10M_mol_token_e5/infer_{data}-chemberta-zinc1m-mol-token-e5-{dataset}-batch{batch_size}-epoch{nb_epoch}-lr{learning_rate}", 
                        dataset_column="smiles", 
                        job_config_id='cjc-13')
                    print(response)


{'job_id': '91edf1b0-f111-4771-947b-7a38ccf09f7a', 'operation_cost': 0.10418099999999998}
{'job_id': '248d3796-1c2d-4431-a87c-0a2cc1b2477d', 'operation_cost': 0.0130515}
{'job_id': 'b79a70d3-93a8-4fcc-9bf7-fbba718cd08f', 'operation_cost': 0.0130515}


In [17]:
for data in ["train", "test", "valid"]:
    for learning_rate in [1e-05]:
        for batch_size in [15]:
            for nb_epoch in [50]:
                pyprithvi.download_file(address=f"chiron://LLNL/bench_10M_mol_token_e5/infer_{data}-chemberta-zinc1m-mol-token-e5-{dataset}-batch{batch_size}-epoch{nb_epoch}-lr{learning_rate}.csv", filename=f"infer_data/infer_{data}-chemberta-zinc10m-mol-token-e5-{dataset}-batch{batch_size}-epoch{nb_epoch}-lr{learning_rate}.csv")

INFO:pyprithvi.data:File download successful!
INFO:pyprithvi.data:File download successful!
INFO:pyprithvi.data:File download successful!


In [23]:
import pandas as pd
data = pd.read_csv(f"../data/finetune_data/normalized_finetune_datasets/norm_{dataset}.csv")
data.head()

,Unnamed: 0,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles,normalized_measured_log_solubility
0,0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,1.088088
1,1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2,-0.119254
2,2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O),0.472487
3,3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,-2.300105
4,4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1,0.820851


In [25]:
# change the column name according to the dataset selected
# BACE - pIC50
# clearance - target
# delaney - measured log solubility in mols per litre
# lipo - exp

mean_value = data['measured log solubility in mols per litre'].mean()
print(mean_value)

std_deviation = data['measured log solubility in mols per litre'].std()
print(std_deviation)

-3.05010195035461
2.096441210089345


In [31]:
dataset_train = dc.data.DiskDataset(f"../data/finetune_data/featurized_molnet_datasets/feat_moleculenet/{dataset}/train")
dataset_valid = dc.data.DiskDataset(f"../data/finetune_data/featurized_molnet_datasets/feat_moleculenet/{dataset}/valid")
dataset_test = dc.data.DiskDataset(f"../data/finetune_data/featurized_molnet_datasets/feat_moleculenet/{dataset}/test")

INFO:deepchem.data.datasets:Loading dataset from disk.
INFO:deepchem.data.datasets:Loading dataset from disk.
INFO:deepchem.data.datasets:Loading dataset from disk.


In [32]:
print(len(dataset_train))
print(len(dataset_valid))
print(len(dataset_test))

902
113
113


In [33]:
y_true_train = dataset_train.y
y_true_valid = dataset_valid.y
y_true_test = dataset_test.y
true_data_dict = {"train": y_true_train, "valid": y_true_valid, "test": y_true_test}

In [35]:
from sklearn.metrics import mean_squared_error
import numpy as np
for data in ["train", "test", "valid"]:
    for learning_rate in [1e-05]:
        for batch_size in [15]:
            for nb_epoch in [50]:
                try:
                    infer_data = pd.read_csv(f"infer_data/infer_{data}-chemberta-zinc10m-mol-token-e5-{dataset}-batch{batch_size}-epoch{nb_epoch}-lr{learning_rate}.csv")
                except:
                    continue
                y_preds = infer_data['y_preds']
                y_true = true_data_dict[data]
                denormalized_y_true = (y_true * std_deviation) + mean_value
                denormalized_y_preds = (y_preds * std_deviation) + mean_value
                rmse_score = np.sqrt(mean_squared_error(denormalized_y_true, denormalized_y_preds))
                print(f"{dataset}_lr{learning_rate}_{data}_batch{batch_size}_epoch{nb_epoch}: {rmse_score}")

                

delaney_lr1e-05_train_batch15_epoch50: 0.49852945772283186
delaney_lr1e-05_test_batch15_epoch50: 1.2073708301406274
delaney_lr1e-05_valid_batch15_epoch50: 1.2097266455161069
